<a href="https://colab.research.google.com/github/VanishaJoshi/Placement-2023/blob/main/loanMLtools.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [8]:
import pandas as pd
from warnings import filterwarnings

In [9]:
filterwarnings("ignore")

In [10]:
train_set=pd.read_csv("/content/Loan-Prediction.csv")
test_set=pd.read_csv("/content/Loan-Prediction.csv")

In [11]:
train_set.head()

,Loan_ID,Gender,Married,Dependents,Education,Self_Employed,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History,Property_Area
0,LP001015,Male,Yes,0,Graduate,No,5720,0,110.0,360.0,1.0,Urban
1,LP001022,Male,Yes,1,Graduate,No,3076,1500,126.0,360.0,1.0,Urban
2,LP001031,Male,Yes,2,Graduate,No,5000,1800,208.0,360.0,1.0,Urban
3,LP001035,Male,Yes,2,Graduate,No,2340,2546,100.0,360.0,NaN,Urban
4,LP001051,Male,No,0,Not Graduate,No,3276,0,78.0,360.0,1.0,Urban


In [12]:
train_set.shape

(367, 12)

In [13]:
train_set.isna().sum()

Loan_ID               0
Gender               11
Married               0
Dependents           10
Education             0
Self_Employed        23
ApplicantIncome       0
CoapplicantIncome     0
LoanAmount            5
Loan_Amount_Term      6
Credit_History       29
Property_Area         0
dtype: int64

# **Data Cleaning**

In [14]:
from MLTools import replacer
replacer(train_set)

In [15]:
train_set.head()

,Loan_ID,Gender,Married,Dependents,Education,Self_Employed,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History,Property_Area
0,LP001015,Male,Yes,0,Graduate,No,5720,0,110.0,360.0,1.000000,Urban
1,LP001022,Male,Yes,1,Graduate,No,3076,1500,126.0,360.0,1.000000,Urban
2,LP001031,Male,Yes,2,Graduate,No,5000,1800,208.0,360.0,1.000000,Urban
3,LP001035,Male,Yes,2,Graduate,No,2340,2546,100.0,360.0,0.825444,Urban
4,LP001051,Male,No,0,Not Graduate,No,3276,0,78.0,360.0,1.000000,Urban


In [18]:
df=train_set.drop(labels=["Loan_ID"],axis=1)

In [19]:
from MLTools import preprocessing
Xnew=preprocessing(df)

# **Machine Leaning Training and Testing**

In [21]:
X=Xnew
Y=train_set['Credit_History']

In [22]:
from sklearn.model_selection import train_test_split
xtrain,xtest,ytrain,ytest=train_test_split(X,Y,test_size=0.2,random_state=21)

In [27]:
#Building The Model : KNN Classification

from sklearn.neighbors import KNeighborsClassifier
from sklearn.preprocessing import LabelEncoder

knc=KNeighborsClassifier(n_neighbors=5)

# Convert ytrain to categorical values if it's continuous
label_encoder = LabelEncoder()
ytrain_encoded = label_encoder.fit_transform(ytrain)

model=knc.fit(xtrain,ytrain_encoded) # Use the encoded ytrain for training

ts_pred= model.predict(xtest.values)
tr_pred= model.predict(xtrain.values)
from sklearn.metrics import accuracy_score,confusion_matrix

# Encode ytest using the same LabelEncoder
ytest_encoded = label_encoder.transform(ytest)

ts_acc=accuracy_score(ytest_encoded,ts_pred)                                  #testing accuracy
tr_acc=accuracy_score(ytrain_encoded,tr_pred)                                 #training acuuracy

In [29]:
from sklearn.metrics import classification_report

# Encode ytest using the same LabelEncoder
ytest_encoded = label_encoder.transform(ytest)

print(classification_report(ytest_encoded,ts_pred)) # Use the encoded ytest for the report

              precision    recall  f1-score   support

           0       1.00      1.00      1.00        13
           1       0.00      0.00      0.00         6
           2       0.90      1.00      0.95        55

    accuracy                           0.92        74
   macro avg       0.63      0.67      0.65        74
weighted avg       0.85      0.92      0.88        74



In [31]:
from sklearn.tree import DecisionTreeClassifier
dtc= DecisionTreeClassifier(random_state=21,criterion="entropy")      #trying out using entropy

# Convert ytrain to categorical values if it's continuous
label_encoder = LabelEncoder()
ytrain_encoded = label_encoder.fit_transform(ytrain)

model=dtc.fit(xtrain,ytrain_encoded) # Use the encoded ytrain for training
tr_pred= model.predict(xtrain)

# Encode ytest using the same LabelEncoder
ytest_encoded = label_encoder.transform(ytest)

ts_pred= model.predict(xtest)
tr_acc=accuracy_score(ytrain_encoded,tr_pred) # Use the encoded ytrain for accuracy calculation
ts_acc=accuracy_score(ytest_encoded,ts_pred) # Use the encoded ytest for accuracy calculation
print(classification_report(ytest_encoded,ts_pred)) # Use the encoded ytest for the report

              precision    recall  f1-score   support

           0       1.00      1.00      1.00        13
           1       1.00      1.00      1.00         6
           2       1.00      1.00      1.00        55

    accuracy                           1.00        74
   macro avg       1.00      1.00      1.00        74
weighted avg       1.00      1.00      1.00        74



In [32]:
tr_acc

1.0

In [33]:
ts_acc

1.0

In [35]:
tg={"max_depth":range(2,20,1),"criterion":['gini','entropy']}
from sklearn.tree import DecisionTreeClassifier
dtc= DecisionTreeClassifier(random_state=21)
from sklearn.model_selection import GridSearchCV

# Ensure Y is a categorical variable
from sklearn.preprocessing import LabelEncoder
label_encoder = LabelEncoder()
Y_encoded = label_encoder.fit_transform(Y)

cv=GridSearchCV(dtc,tg,scoring="recall",cv=4)
cvmodel=cv.fit(X,Y_encoded) # Use the encoded Y for training
cvmodel.best_params_

{'criterion': 'gini', 'max_depth': 2}

In [37]:
dtc= DecisionTreeClassifier(random_state=21,criterion="gini",max_depth=2)
# Convert ytrain to categorical values if it's continuous
from sklearn.preprocessing import LabelEncoder
label_encoder = LabelEncoder()
ytrain_encoded = label_encoder.fit_transform(ytrain)

model=dtc.fit(xtrain,ytrain_encoded) # Use the encoded ytrain for training
tr_pred= model.predict(xtrain)

# Encode ytest using the same LabelEncoder
ytest_encoded = label_encoder.transform(ytest)

ts_pred= model.predict(xtest)
tr_acc=accuracy_score(ytrain_encoded,tr_pred) # Use the encoded ytrain for accuracy calculation
ts_acc=accuracy_score(ytest_encoded,ts_pred) # Use the encoded ytest for accuracy calculation
print(classification_report(ytest_encoded,ts_pred)) # Use the encoded ytest for the report

              precision    recall  f1-score   support

           0       1.00      1.00      1.00        13
           1       1.00      1.00      1.00         6
           2       1.00      1.00      1.00        55

    accuracy                           1.00        74
   macro avg       1.00      1.00      1.00        74
weighted avg       1.00      1.00      1.00        74



In [39]:
tg={"min_samples_leaf":range(2,20,1),"criterion":['gini','entropy']}
from sklearn.tree import DecisionTreeClassifier
dtc= DecisionTreeClassifier(random_state=21)
from sklearn.model_selection import GridSearchCV

# Ensure Y is a categorical variable
from sklearn.preprocessing import LabelEncoder
label_encoder = LabelEncoder()
Y_encoded = label_encoder.fit_transform(Y)

cv=GridSearchCV(dtc,tg,scoring="recall",cv=4)
cvmodel=cv.fit(X,Y_encoded) # Use the encoded Y values
cvmodel.best_params_

{'criterion': 'gini', 'min_samples_leaf': 2}

In [41]:
dtc= DecisionTreeClassifier(criterion="gini",min_samples_leaf=2)

# Convert ytrain to categorical values if it's continuous
from sklearn.preprocessing import LabelEncoder
label_encoder = LabelEncoder()
ytrain_encoded = label_encoder.fit_transform(ytrain)

model=dtc.fit(xtrain,ytrain_encoded) # Use the encoded ytrain for training
tr_pred= model.predict(xtrain)

# Encode ytest using the same LabelEncoder
ytest_encoded = label_encoder.transform(ytest)

ts_pred= model.predict(xtest)
tr_acc=accuracy_score(ytrain_encoded,tr_pred) # Use the encoded ytrain for accuracy calculation
ts_acc=accuracy_score(ytest_encoded,ts_pred) # Use the encoded ytest for accuracy calculation
print(classification_report(ytest_encoded,ts_pred)) # Use the encoded ytest for the report

              precision    recall  f1-score   support

           0       1.00      1.00      1.00        13
           1       1.00      1.00      1.00         6
           2       1.00      1.00      1.00        55

    accuracy                           1.00        74
   macro avg       1.00      1.00      1.00        74
weighted avg       1.00      1.00      1.00        74



In [43]:
dtc= DecisionTreeClassifier(random_state=21,criterion="gini",min_samples_split=2)

# Convert ytrain to categorical values if it's continuous
from sklearn.preprocessing import LabelEncoder
label_encoder = LabelEncoder()
ytrain_encoded = label_encoder.fit_transform(ytrain)

model=dtc.fit(xtrain,ytrain_encoded) # Use the encoded ytrain values

tr_pred= model.predict(xtrain)

# Encode ytest using the same LabelEncoder
ytest_encoded = label_encoder.transform(ytest)

ts_pred= model.predict(xtest)
tr_acc=accuracy_score(ytrain_encoded,tr_pred) # Use the encoded ytrain values
ts_acc=accuracy_score(ytest_encoded,ts_pred) # Use the encoded ytest values
print(classification_report(ytest_encoded,ts_pred)) # Use the encoded ytest values

              precision    recall  f1-score   support

           0       1.00      1.00      1.00        13
           1       1.00      1.00      1.00         6
           2       1.00      1.00      1.00        55

    accuracy                           1.00        74
   macro avg       1.00      1.00      1.00        74
weighted avg       1.00      1.00      1.00        74



# **Decision tree**

In [45]:
from sklearn.ensemble import RandomForestClassifier
rfc=RandomForestClassifier(random_state=21,n_estimators=20)  #trying out n_estimators as 20

# Convert ytrain to categorical values if it's continuous
from sklearn.preprocessing import LabelEncoder
label_encoder = LabelEncoder()
ytrain_encoded = label_encoder.fit_transform(ytrain)

model=rfc.fit(xtrain,ytrain_encoded) # Use the encoded ytrain for training
tr_pred=model.predict(xtrain)

# Encode ytest using the same LabelEncoder
ytest_encoded = label_encoder.transform(ytest)

ts_pred=model.predict(xtest)
tr_acc=accuracy_score(ytrain_encoded,tr_pred) # Use the encoded ytrain for accuracy calculation
ts_acc=accuracy_score(ytest_encoded,ts_pred) # Use the encoded ytest for accuracy calculation
print(classification_report(ytest_encoded,ts_pred)) # Use the encoded ytest for the report

              precision    recall  f1-score   support

           0       1.00      1.00      1.00        13
           1       1.00      1.00      1.00         6
           2       1.00      1.00      1.00        55

    accuracy                           1.00        74
   macro avg       1.00      1.00      1.00        74
weighted avg       1.00      1.00      1.00        74



In [47]:
tg={"n_estimators":range(2,30,1),"max_depth":range(2,20)}
rfc=RandomForestClassifier(random_state=21)
from sklearn.model_selection import GridSearchCV

# Convert Y to categorical values if it's continuous
from sklearn.preprocessing import LabelEncoder
label_encoder = LabelEncoder()
Y_encoded = label_encoder.fit_transform(Y)

cv=GridSearchCV(rfc,tg,scoring="recall",cv=4)
cvmodel=cv.fit(X,Y_encoded) # Use the encoded Y for training
cvmodel.best_params_

{'max_depth': 2, 'n_estimators': 2}

# **Random Forest Classifier**

In [49]:
from sklearn.ensemble import RandomForestClassifier
rfc=RandomForestClassifier(random_state=21,n_estimators=2,max_depth=2)

# Convert ytrain to categorical values if it's continuous
from sklearn.preprocessing import LabelEncoder
label_encoder = LabelEncoder()
ytrain_encoded = label_encoder.fit_transform(ytrain)

model=rfc.fit(xtrain,ytrain_encoded) # Use the encoded ytrain for training
tr_pred=model.predict(xtrain)

# Encode ytest using the same LabelEncoder
ytest_encoded = label_encoder.transform(ytest)

ts_pred=model.predict(xtest)
tr_acc=accuracy_score(ytrain_encoded,tr_pred) # Use the encoded ytrain for accuracy calculation
ts_acc=accuracy_score(ytest_encoded,ts_pred) # Use the encoded ytest for accuracy calculation
print(classification_report(ytest_encoded,ts_pred)) # Use the encoded ytest for the report

              precision    recall  f1-score   support

           0       0.71      0.92      0.80        13
           1       0.00      0.00      0.00         6
           2       0.96      1.00      0.98        55

    accuracy                           0.91        74
   macro avg       0.56      0.64      0.59        74
weighted avg       0.84      0.91      0.87        74



In [51]:
confusion_matrix(ytest_encoded,ts_pred) # Use the encoded ytest for the confusion matrix

array([[12,  0,  1],
       [ 5,  0,  1],
       [ 0,  0, 55]])

In [52]:
test_df=test_set.drop('Loan_ID',axis=1)

In [53]:
test_X = preprocessing(test_df)

In [54]:
test_X.shape

(367, 20)

In [55]:
train_X=X

In [56]:
train_X.shape

(367, 20)

# **Prediction and Testing**

In [58]:
!pip install scikit-learn
import pandas as pd
from sklearn.impute import SimpleImputer
# Create an imputer object with your desired strategy (e.g., mean, median, most_frequent)
imputer = SimpleImputer(strategy='mean')

# Fit the imputer on your training data (if you have a separate training set)
# imputer.fit(train_X)

# Transform the test data to fill in missing values
test_X_imputed = pd.DataFrame(imputer.fit_transform(test_X), columns=test_X.columns)

# Now you can use the imputed data for prediction
S = model.predict(test_X_imputed)

In [59]:
Solution=pd.DataFrame(test_set['Loan_ID'])

In [60]:
Solution['Loan_Status']=S

In [61]:
Solution

,Loan_ID,Loan_Status
0,LP001015,2
1,LP001022,2
2,LP001031,2
3,LP001035,0
4,LP001051,2
...,...,...
362,LP002971,2
363,LP002975,2
364,LP002980,0
365,LP002986,2


## **Adding Predicted Value column to our dataset**

In [64]:
Solution.to_csv("/content/Loan-Prediction.csv")

In [63]:
Predictied_Set=pd.DataFrame(test_set)

In [65]:
Predictied_Set['Loan_Status']=S

In [66]:
Predictied_Set

,Loan_ID,Gender,Married,Dependents,Education,Self_Employed,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History,Property_Area,Loan_Status
0,LP001015,Male,Yes,0,Graduate,No,5720,0,110.0,360.0,1.0,Urban,2
1,LP001022,Male,Yes,1,Graduate,No,3076,1500,126.0,360.0,1.0,Urban,2
2,LP001031,Male,Yes,2,Graduate,No,5000,1800,208.0,360.0,1.0,Urban,2
3,LP001035,Male,Yes,2,Graduate,No,2340,2546,100.0,360.0,NaN,Urban,0
4,LP001051,Male,No,0,Not Graduate,No,3276,0,78.0,360.0,1.0,Urban,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...
362,LP002971,Male,Yes,3+,Not Graduate,Yes,4009,1777,113.0,360.0,1.0,Urban,2
363,LP002975,Male,Yes,0,Graduate,No,4158,709,115.0,360.0,1.0,Urban,2
364,LP002980,Male,No,0,Graduate,No,3250,1993,126.0,360.0,NaN,Semiurban,0
365,LP002986,Male,Yes,0,Graduate,No,5000,2393,158.0,360.0,1.0,Rural,2
